In [1]:
import torch
import torch.nn as nn
import numpy as np
import data_prep
from lstm import *
from trainer import Optim
import time
import importlib

In [2]:
params = {}
params['optim'] = 'adam'
params['seq_in_len'] = 20
params['seq_out_len'] = 15
params['lr'] = 0.001
params['weight_decay'] = 0.0001
params['clip'] = 5
params['data'] = '.\SlowVaryingAmplitudes_23_30.csv'
params['device'] = 'cuda'
params['L1Loss'] = True
params['epochs'] = 500
params['cl']= False
params['cl_step_size']=10
params['batch_size']=16
params['save'] = './models/best_lstm_model_all_amplitudes_.pt'

In [3]:
device = torch.device(params['device'])
torch.set_num_threads(3)

In [4]:
importlib.reload(data_prep)
Data = data_prep.DataLoaderMulti(params['data'], 0.6, params['device'], params['seq_in_len'], params['seq_out_len'])
mean = Data.train[2]
std = Data.train[3]

(1, 33)
The dataset shape is: (538, 33)
The input sequence length is: 20
The output sequence length is: 15
The train X-dataset shape is: torch.Size([287, 20, 33])
The validation X-dataset shape is: torch.Size([181, 20, 33])

The train y-dataset shape is: torch.Size([287, 15, 33])
The validation y-dataset shape is: torch.Size([181, 15, 33])



In [5]:
input_size = Data.train[0].shape[2]
hidden_size = 200
output_size = Data.train[0].shape[2]
num_layers = 3

In [6]:
model = LSTMModel(input_size, hidden_size, output_size, num_layers, params['seq_out_len'])
model = model.to(device)

In [7]:
nParams = sum([p.nelement() for p in model.parameters()])
print('Number of model parameters is', nParams, flush=True)

Number of model parameters is 930695


In [8]:
def train(data, X, Y, model, criterion, lr, wd, batch_size, clip, mean, std, cl, cl_step_size, seq_out_len):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    model.train()
    optimizer.zero_grad()
    total_loss = 0
    n_samples = 0
    task_level = 0
    
    iter = 0
    for X, Y in data.get_batches(X, Y, batch_size, True):
        model.zero_grad()
        output = model(X)
        # print(output.shape)
        predictions = mean + output * std
        real_values = mean + Y * std

        if iter%cl_step_size==0 and task_level<=seq_out_len:
            task_level +=1
        if cl:
            loss = criterion(predictions[:, :, :task_level], real_values[:, :, :task_level])
        else:
            loss = criterion(predictions, real_values)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            
        total_loss += loss.item()
        n_samples += (output.size(0) * data.m)
        grad_norm = optim.step()

        # if iter%100==0:
        #     print('iter:{:3d} | loss: {:.3f}'.format(iter,loss.item()/(output.size(0) * data.m)))
        iter += 1
        # print(f"The number of batches used are:{iter}")
        
    return total_loss / n_samples

In [9]:
def evaluate(data, X, Y, model, evaluateL2, evaluateL1, batch_size, mean, std):
    model.eval()
    total_loss = 0
    total_loss_l1 = 0
    n_samples = 0
    predict = None
    test = None

    for X, Y in data.get_batches(X, Y, batch_size, False):
        with torch.no_grad():
            output = model(X)
        if predict is None:
            predict = output
            test = Y
        else:
            predict = torch.cat((predict, output))
            test = torch.cat((test, Y))

        total_loss += evaluateL2(mean + output * std, mean + Y * std).item()
        total_loss_l1 += evaluateL1(mean + output * std, mean + Y * std).item()
        n_samples += (output.size(0) * data.m)

    rse = math.sqrt(total_loss / n_samples) / data.rse
    rae = (total_loss_l1 / n_samples) / data.rae

    predict = (predict).data.cpu().numpy()
    Ytest = test.data.cpu().numpy()
    sigma_p = (predict).std(axis=0)
    sigma_g = (Ytest).std(axis=0)
    mean_p = predict.mean(axis=0)
    mean_g = Ytest.mean(axis=0)
    index = (sigma_g != 0)
    correlation = ((predict - mean_p) * (Ytest - mean_g)).mean(axis=0) / (sigma_p * sigma_g)
    correlation = (correlation[index]).mean()
    return rse, rae, correlation, predict

In [10]:
if params['L1Loss']:
    criterion = nn.L1Loss(reduction="sum").to(device)
else:
    criterion = nn.MSELoss(reduction="sum").to(device)

evaluateL2 = nn.MSELoss(reduction="sum").to(device)
evaluateL1 = nn.L1Loss(reduction="sum").to(device)

In [11]:
best_val = 10000000
optim = Optim(model.parameters(), params['optim'], params['lr'], params['clip'], lr_decay=params['weight_decay'])

In [12]:
# At any point you can hit Ctrl + C to break out of training early.
import math

try:
    print('begin training')
    for epoch in range(1, params['epochs'] + 1):
        epoch_start_time = time.time()
        # train_loss = train(Data, Data.train[0], Data.train[1], model, criterion, optim, args.batch_size, args.clip, torch.Tensor(mean).to(device), torch.Tensor(std).to(device))
        train_loss = train(Data, Data.train[0], Data.train[1], model, criterion, params['lr'], params['weight_decay'], params['batch_size'], params['clip'], torch.Tensor(mean).to(device), torch.Tensor(std).to(device), params['cl'], params['cl_step_size'], params['seq_out_len'])
        val_loss, val_rae, val_corr, val_predict = evaluate(Data, Data.valid[0], Data.valid[1], model, evaluateL2, evaluateL1, params['batch_size'], torch.Tensor(mean).to(device), torch.Tensor(std).to(device))
        if epoch % 50 == 0:
            print('| end of epoch {:3d} | time: {:5.2f}s | train_loss {:5.4f} | valid rse {:5.4f} | valid rae {:5.4f} | valid corr  {:5.4f}'.format(
                    epoch, (time.time() - epoch_start_time), train_loss, val_loss, val_rae, val_corr), flush=True)
        # Save the model if the validation loss is the best we've seen so far.

        if val_loss < best_val:
            with open(params['save'], 'wb') as f:
                torch.save(model, f)
            best_val = val_loss
        # if epoch % 50 == 0:
        #     test_acc, test_rae, test_corr, predictions = evaluate(Data, Data.test[0], Data.test[1], model, evaluateL2, evaluateL1, args.batch_size)
        #     print("test rse {:5.4f} | test rae {:5.4f} | test corr {:5.4f}".format(test_acc, test_rae, test_corr), flush=True)

except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

begin training
| end of epoch  50 | time:  0.12s | train_loss 0.0125 | valid rse 1.5589 | valid rae 6.2503 | valid corr  0.1876
| end of epoch 100 | time:  0.12s | train_loss 0.0077 | valid rse 1.6695 | valid rae 6.7981 | valid corr  0.1148
| end of epoch 150 | time:  0.12s | train_loss 0.0069 | valid rse 1.7117 | valid rae 6.9370 | valid corr  0.0914
| end of epoch 200 | time:  0.12s | train_loss 0.0059 | valid rse 1.7467 | valid rae 7.0777 | valid corr  0.0909
| end of epoch 250 | time:  0.13s | train_loss 0.0046 | valid rse 1.7440 | valid rae 7.0712 | valid corr  0.0847
| end of epoch 300 | time:  0.12s | train_loss 0.0044 | valid rse 1.7481 | valid rae 7.1031 | valid corr  0.0940
| end of epoch 350 | time:  0.12s | train_loss 0.0043 | valid rse 1.7435 | valid rae 7.0892 | valid corr  0.0898
| end of epoch 400 | time:  0.13s | train_loss 0.0047 | valid rse 1.7370 | valid rae 7.0677 | valid corr  0.0784
| end of epoch 450 | time:  0.12s | train_loss 0.0046 | valid rse 1.7403 | valid 

In [13]:
errors = []

vacc = []
vrae = []
vcorr = []
# acc = []
# rae = []
# corr = []
for i in range(10):
    with open(params['save'], 'rb') as f:
        model = torch.load(f)

    vtest_acc, vtest_rae, vtest_corr, val_predictions = evaluate(Data, Data.valid[0], Data.valid[1], model, evaluateL2, evaluateL1, params['batch_size'], torch.Tensor(mean).to(device), torch.Tensor(std).to(device))
    # test_acc, test_rae, test_corr, test_predictions = evaluate(Data, Data.test[0], Data.test[1], model, evaluateL2, evaluateL1, args.batch_size)

    print("validation rse {:5.4f} | validation rae {:5.4f} | validation corr {:5.4f}".format(vtest_acc, vtest_rae, vtest_corr))
    # print("final test rse {:5.4f} | test rae {:5.4f} | test corr {:5.4f}\n".format(test_acc, test_rae, test_corr))
    
    # val_acc, val_rae, val_corr, test_acc, test_rae, test_corr = vtest_acc, vtest_rae, vtest_corr, test_acc, test_rae, test_corr
    val_acc, val_rae, val_corr = vtest_acc, vtest_rae, vtest_corr
    vacc.append(val_acc)
    vrae.append(val_rae)
    vcorr.append(val_corr)
    # acc.append(test_acc)
    # rae.append(test_rae)
    # corr.append(test_corr)
print('\n\n')
print('10 runs average')
print('\n\n')
print("valid\trse\trae\tcorr")
print("mean\t{:5.4f}\t{:5.4f}\t{:5.4f}".format(np.mean(vacc), np.mean(vrae), np.mean(vcorr)))
print("std\t{:5.4f}\t{:5.4f}\t{:5.4f}".format(np.std(vacc), np.std(vrae), np.std(vcorr)))
print('\n\n')

errors.extend([np.mean(vacc), np.mean(vrae), np.mean(vcorr)])
np.savetxt('LSTM.csv', errors, delimiter=',')
# print("test\trse\trae\tcorr")
# print("mean\t{:5.4f}\t{:5.4f}\t{:5.4f}".format(np.mean(acc), np.mean(rae), np.mean(corr)))
# print("std\t{:5.4f}\t{:5.4f}\t{:5.4f}".format(np.std(acc), np.std(rae), np.std(corr)))

validation rse 1.3874 | validation rae 5.5707 | validation corr 0.0468
validation rse 1.3874 | validation rae 5.5707 | validation corr 0.0468
validation rse 1.3874 | validation rae 5.5707 | validation corr 0.0468
validation rse 1.3874 | validation rae 5.5707 | validation corr 0.0468
validation rse 1.3874 | validation rae 5.5707 | validation corr 0.0468
validation rse 1.3874 | validation rae 5.5707 | validation corr 0.0468
validation rse 1.3874 | validation rae 5.5707 | validation corr 0.0468


C:\Users\samar\AppData\Local\Temp\ipykernel_12712\2136362194.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f)


validation rse 1.3874 | validation rae 5.5707 | validation corr 0.0468
validation rse 1.3874 | validation rae 5.5707 | validation corr 0.0468
validation rse 1.3874 | validation rae 5.5707 | validation corr 0.0468



10 runs average



valid	rse	rae	corr
mean	1.3874	5.5707	0.0468
std	0.0000	0.0000	0.0000





In [14]:
import matplotlib.pyplot as plt
import random

fig, axes = plt.subplots(11,3, figsize=[35,25], dpi=300, sharex = True)
fig.suptitle("Multi step ahead predictions", fontsize=40)
plt.rcParams["font.family"]="serif"
plt.rcParams['font.size']=30

random_dataset = random.randint(0,Data.valid[0].shape[0])
for i in range(Data.valid[0].shape[2]):
    j = i//3
    k = i%3
    x_range = np.arange(0,len(Data.valid[0][random_dataset,:,i]))
    y_range = np.arange(len(Data.valid[0][random_dataset,:,i]),len(Data.valid[0][random_dataset,:,i])+len(Data.valid[1][random_dataset,:,i]))
    axes[j,k].plot(x_range, Data.valid[0][random_dataset,:,i], 'k-.',linewidth=2, label="x-data")
    axes[j,k].plot(y_range, Data.valid[1][random_dataset,:,i], color='b', marker = 'h', markersize = 3, label="truth")
    axes[j,k].plot(y_range, val_predictions[random_dataset,:,i], color='r', marker = 'h', markersize = 3, label='prediction')
    axes[j,k].set_title(f'a{i}')
    axes[j,k].tick_params(axis='both', which='major', labelsize=18)

    axes[j,k].legend(ncol = 3, fontsize = 14)

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.88, 
                    wspace=0.4, 
                    hspace=0.4)    
# plt.savefig(f"multi step ahead predictions_during training_num_layers_{args.layers}.png")
plt.show()